In [ ]:
import numpy as np 
import pandas as pd 
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
root='/kaggle/input/paribahan-bd/Local-Vehicles/Local-Vehicles/'
data={}
for class_ in os.listdir(root):
    for image in os.listdir(root+class_):
        data[root+class_+'/'+image]=class_
        
root='/kaggle/input/paribahan-bd/generated_images/'
for class_ in os.listdir(root):
    for image in os.listdir(root+class_):
        data[root+class_+'/'+image]=class_

In [ ]:
data=pd.DataFrame(data.items(),columns=['file','type'])
data=data.sample(frac=1)
data

In [ ]:
top=data['type'].value_counts()[:6].keys()
data = data[data['type'].isin(top)]

In [ ]:
from matplotlib import pyplot as plt
_,_,autotexts=plt.pie(data["type"].value_counts(), labels=data['type'].unique() , autopct='')
plt.axis('equal')
# plt.title('data distribution')
for autotext in autotexts:
    autotext.set_visible(False)

In [ ]:
data

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
data['type']=labelencoder.fit_transform(data['type'])

In [ ]:
import pickle
output = open('encoder.pkl', 'wb')
pickle.dump(labelencoder, output)
output.close()

In [ ]:
def split_data(data,ratio):
    last=int(len(data)*ratio)
    return data[:last], data[last:]
train,test=split_data(data,.08)

In [ ]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import pandas as pd
from PIL import Image
size=224
batch_size = 32
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx, 0]
        image = Image.open(img_name).convert('RGB')
        label = self.dataframe.iloc[idx, -1]
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [ ]:
batch_size=32
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(32),
    transforms.CenterCrop(32),
    transforms.Resize((size, size)),
    transforms.Normalize(mean=[0.0, 0.0, 0.0], std=[1.0, 1.0, 1.0]),
])

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((size, size)),
    transforms.Normalize(mean=[0.0, 0.0, 0.0], std=[1.0, 1.0, 1.0]),
])

custom_dataset_train = CustomDataset(train, transform=transform_train)
data_loader_train = DataLoader(custom_dataset_train, batch_size=batch_size, shuffle=True)

custom_dataset_test = CustomDataset(test, transform=transform)
data_loader_test = DataLoader(custom_dataset_test, batch_size=1, shuffle=False)

In [ ]:
class FineTunedVGG(nn.Module):
    def __init__(self, num_classes, input_size=224):
        super(FineTunedVGG, self).__init__()
        self.vgg = models.vgg16(pretrained=True)
        
        self.st = 8
        self.blocks = []
        for param in self.vgg.parameters():
            param.requires_grad = False
        
        x = torch.randn(1, 3, input_size, input_size)
        for idx, layer in enumerate(self.vgg.features):
            if isinstance(layer, nn.Conv2d):
                x = layer(x)
                if idx in [12, 22, 32]:  
                    self.blocks.append(x)
        
        
        for idx, block in enumerate(self.blocks):
        
            filters = block.size(1)
            depthwise_conv = nn.Conv2d(filters, filters, kernel_size=3, padding=1, groups=filters)
            depthwise_sep_conv = nn.Conv2d(filters, 128, kernel_size=1, padding=0)
            bn = nn.BatchNorm2d(128)
            pooled_block = nn.MaxPool2d(kernel_size=self.st, stride=self.st)
            self.st = self.st // 2
            self.blocks[idx] = nn.Sequential(depthwise_conv, depthwise_sep_conv, bn, pooled_block)
        
        self.vgg.add_module('ConcatenatedBlocks', nn.Sequential(*self.blocks))
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(1000, num_classes) 
    
    def forward(self, x):
        x = self.vgg(x)
#         x=self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

model = FineTunedVGG(num_classes=6, input_size=224) 


In [ ]:
num_epochs = 200
model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

i=0

for epoch in range(num_epochs):
    for images, labels in data_loader_train:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        print(i,end=' ')
        i+=1
        
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

print("Training finished.")

In [ ]:
torch.save(model,'model.pth')

In [ ]:

predictions = []
ground_truth = []

with torch.no_grad():  
    for inputs, targets in data_loader_test:
        
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model(inputs)
        
        predictions.extend(outputs.argmax(dim=1).cpu().numpy())
        ground_truth.extend(targets.cpu().numpy())


predictions = np.array(predictions)
ground_truth = np.array(ground_truth)


In [ ]:
# pkl_file = open('encoder.pkl', 'rb')
# labelencoder = pickle.load(pkl_file) 
# pkl_file.close()

In [ ]:
ground_truth=labelencoder.inverse_transform(ground_truth)
predictions=labelencoder.inverse_transform(predictions)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, matthews_corrcoef

print("acc ",accuracy_score(predictions, ground_truth))
print("preci ",precision_score(predictions, ground_truth, average='weighted'))
print("recall ",recall_score(predictions, ground_truth,average='weighted'))
print("f1 ",f1_score(predictions, ground_truth,average='weighted'))
print("mcc ",matthews_corrcoef(predictions, ground_truth))

In [ ]:
# model=torch.load('model.pth')

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
true=ground_truth
predicted=predictions
class_=labelencoder.inverse_transform(test['type'].value_counts().keys())
cm = confusion_matrix(true, predicted)
fig = plt.figure(figsize=(6, 5))
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax,cmap="Blues", fmt='g'); 
ax.set_xlabel('Predicted', fontsize=12)
ax.xaxis.set_label_position('bottom')
plt.xticks(rotation=90)
ax.xaxis.set_ticklabels(class_, fontsize = 10)
ax.xaxis.tick_bottom()

ax.set_ylabel('True', fontsize=12)
ax.yaxis.set_ticklabels(class_, fontsize = 10)
plt.yticks(rotation=0)

plt.title('', fontsize=60)

plt.savefig('ConMat24.png')
plt.show()